In [1]:
import librosa
import os

In [2]:
import mir_eval

In [3]:
from collections import OrderedDict

In [4]:
import pandas as pd
import numpy as np
np.set_printoptions(precision=3)
pd.set_option('precision', 4, "display.max_rows", 999)

In [5]:
def make_beatles_corpus(iso_path):
    
    # Beat files
    beat_path = os.path.join(iso_path, 'beat')
    
    annotations = librosa.util.find_files(beat_path, ext='txt')
    
    audio = [ann.replace('/beat/', '/audio/').replace('.txt', '.flac') for ann in annotations]
    
    
    data = []
    for aud, ann in zip(audio, annotations):
        if os.path.exists(aud) and os.path.exists(ann):
            data.append((aud, ann))
    
    return pd.DataFrame(data=data, columns=['audio', 'annotation'])

In [6]:
def make_smc_data(smc_path):
    
    # Beat files
    beat_path = os.path.join(smc_path, 'SMC_MIREX_Annotations')
    annotations = librosa.util.find_files(beat_path, ext='txt')
    
    # Audio files
    audio_path = os.path.join(smc_path, 'SMC_MIREX_Audio')
    audio = librosa.util.find_files(audio_path, ext='wav')
    
    data = zip(audio, annotations)
    
    return pd.DataFrame(data=data, columns=['audio', 'annotation'])

In [7]:
def make_output_path(base, outpath):
    
    root = os.path.splitext(base)[0]
    
    output = os.path.join(outpath, os.path.extsep.join([root, 'json']))
    
    return output

In [8]:
def analyze(dframe, outpath='/home/bmcfee/git/librosa_parameters/data/beat/'):
    
    index = dframe.index[0]
    
    base = os.path.basename(dframe['audio'][index])
    
    outfile = make_output_path(base, outpath)
    
    if os.path.exists(outfile):
        print 'Cached {}'.format(base)
        data = pd.read_json(outfile, orient='records')
        return data
    else:
        print 'Processing {}'.format(base)
    
    # Load the truth
    ref_times = pd.read_table(dframe['annotation'][index], header=None, sep='\s+',
                              usecols=[0], error_bad_lines=False)[0].values

    # Load the audio
    sr = 22050
    y, _ = librosa.load(dframe['audio'][index], None)
    y = librosa.resample(y, _, sr, res_type='sinc_fastest')
    
    # Construct the output container
    results = []
    
    # Onset strength parameters
    
    for fmax in [8000, 11025]:
        for n_mels in [32, 64, 128]:
            S = librosa.feature.melspectrogram(y=y, sr=sr, fmax=fmax, n_mels=n_mels)
            S = librosa.logamplitude(S)
            
            for aggregate in [np.mean, np.median]:
                # Compute the onset detection function
                oenv = librosa.onset.onset_strength(S=S, sr=sr,
                                                    aggregate=aggregate)
                
                    # Tempo estimator parameters
                for ac_size in [2, 4, 8]:
                    for std_bpm in [0.5, 1.0, 2.0]:
                        tempo = librosa.beat.estimate_tempo(oenv,
                                                                    sr=sr,
                                                                    ac_size=ac_size,
                                                                    std_bpm=std_bpm)
                                
                        for tightness in [50, 100, 400]:
                            # Evaluate the predictions
                            params = {'aggregate': aggregate.__name__,
                                              'fmax': fmax,
                                              'n_mels': n_mels,
                                              'ac_size': ac_size,
                                              'std_bpm': std_bpm,
                                              'tightness': tightness}
                            
                            _, beats = librosa.beat.beat_track(sr=sr,
                                                                       onset_envelope=oenv,
                                                                       trim=False, 
                                                                       tightness=tightness,
                                                                       bpm=tempo)
                                    
                            est_times = librosa.frames_to_time(beats, sr=sr)
                            scores = mir_eval.beat.evaluate(ref_times, est_times)
                            
                            cont = OrderedDict(index=index)
                            cont.update(params)
                            cont.update(scores)
                            results.append(cont)
    # Blow away the cache
    #librosa.cache.clear()
    data = pd.DataFrame.from_dict(results, orient='columns')
    data.to_json(outfile, orient='records')
        
    return data

In [9]:
def analyze_corpus(corpus):
    
    results = None
    for idx in corpus.index:
        new_results = analyze(corpus.loc[[idx]])
        if results is None:
            results = new_results
        else:
            results = pd.concat([results, new_results])
            
    return results

In [10]:
from joblib import Parallel, delayed

In [11]:
def p_analyze_corpus(corpus, n_jobs=3):
    
    results = None
    
    dfunc = delayed(analyze)
    
    results = Parallel(n_jobs=n_jobs, verbose=10)(dfunc(corpus.loc[[idx]])
                                                  for idx in corpus.index)

    return pd.concat(results)

---

In [12]:
smc_data = make_smc_data('/home/bmcfee/data/SMC_Mirex/')

In [13]:
smc_results = p_analyze_corpus(smc_data)

[Parallel(n_jobs=3)]: Done   1 out of  35 | elapsed:    0.1s remaining:    2.8s
[Parallel(n_jobs=3)]: Done  21 out of 217 | elapsed:  3.6min remaining: 33.6min
[Parallel(n_jobs=3)]: Done  43 out of 217 | elapsed:  6.5min remaining: 26.4min
[Parallel(n_jobs=3)]: Done  65 out of 217 | elapsed:  9.0min remaining: 21.0min
[Parallel(n_jobs=3)]: Done  87 out of 217 | elapsed: 11.7min remaining: 17.4min
[Parallel(n_jobs=3)]: Done 109 out of 217 | elapsed: 14.6min remaining: 14.5min
[Parallel(n_jobs=3)]: Done 131 out of 217 | elapsed: 17.2min remaining: 11.3min
[Parallel(n_jobs=3)]: Done 153 out of 217 | elapsed: 19.9min remaining:  8.3min
[Parallel(n_jobs=3)]: Done 175 out of 217 | elapsed: 22.6min remaining:  5.4min
[Parallel(n_jobs=3)]: Done 197 out of 217 | elapsed: 25.6min remaining:  2.6min
[Parallel(n_jobs=3)]: Done 217 out of 217 | elapsed: 28.2min finished


Cached SMC_002.wav
Cached SMC_001.wav
Cached SMC_003.wav
Cached SMC_004.wav
Cached SMC_005.wav
Cached SMC_006.wav
Processing SMC_009.wavProcessing SMC_007.wav
Processing SMC_008.wav

Processing SMC_010.wavProcessing SMC_012.wavProcessing SMC_011.wav


Processing SMC_013.wavProcessing SMC_015.wavProcessing SMC_014.wav


Processing SMC_016.wavProcessing SMC_018.wavProcessing SMC_017.wav


Processing SMC_019.wavProcessing SMC_022.wavProcessing SMC_021.wav


Processing SMC_023.wavProcessing SMC_026.wavProcessing SMC_024.wav


Processing SMC_027.wavProcessing SMC_030.wavProcessing SMC_028.wav


Processing SMC_032.wavProcessing SMC_034.wavProcessing SMC_033.wav


Processing SMC_036.wavProcessing SMC_037.wavProcessing SMC_035.wav


Processing SMC_041.wavProcessing SMC_043.wavProcessing SMC_038.wav


Processing SMC_044.wavProcessing SMC_047.wavProcessing SMC_042.wav


Processing SMC_048.wavProcessing SMC_052.wavProcessing SMC_046.wav


Processing SMC_054.wavProcessing SMC_056.wavProcessing SMC

In [14]:
smc_results.to_json('/home/bmcfee/git/librosa_parameters/smc_log_results.json', orient='records')

In [12]:
smc_results = pd.read_json('/home/bmcfee/git/librosa_parameters/smc_log_results.json', orient='records')

In [13]:
smc_scores = smc_results.groupby(['aggregate', 'fmax', 'n_mels', 'ac_size', 'std_bpm', 'tightness']).mean()

In [14]:
best_igain = smc_scores['Information gain'].argmax()

In [15]:
best_fmeas = smc_scores['F-measure'].argmax()

In [16]:
best_amlt = smc_scores['Any Metric Level Total'].argmax()

In [17]:
best_cmlt = smc_scores['Correct Metric Level Total'].argmax()

In [18]:
smc_scores.loc[[best_igain, best_fmeas, best_amlt, best_cmlt]]

Any Metric Level Continuous  \
aggregate fmax n_mels ac_size std_bpm tightness                                
median    8000 128    8       2       100                              0.173   
mean      8000 128    2       2       50                               0.137   
median    8000 128    8       1       50                               0.155   
                      2       1       100                              0.163   

                                                 Any Metric Level Total  \
aggregate fmax n_mels ac_size std_bpm tightness                           
median    8000 128    8       2       100                         0.316   
mean      8000 128    2       2       50                          0.315   
median    8000 128    8       1       50                          0.334   
                      2       1       100                         0.328   

                                                 Cemgil  \
aggregate fmax n_mels ac_size std_bpm tightness           
median    8000 128    8       2       100         0.238   
mean      8000 128    2       2       50          0.249   
median    8000 128    8       1       50          0.243   
                      2       1       100         0.240   

                                                 Cemgil Best Metric Level  \
aggregate fmax n_mels ac_size std_bpm tightness                             
median    8000 128    8       2       100                           0.305   
mean      8000 128    2       2       50                            0.321   
median    8000 128    8       1       50                            0.303   
                      2       1       100                           0.299   

                                                 Correct Metric Level Continuous  \
aggregate fmax n_mels ac_size std_bpm tightness                                    
median    8000 128    8       2       100                                  0.105   
mean      8000 128    2       2       50                                   0.072   
median    8000 128    8       1       50                                   0.097   
                      2       1       100                                  0.107   

                                                 Correct Metric Level Total  \
aggregate fmax n_mels ac_size std_bpm tightness                               
median    8000 128    8       2       100                             0.172   
mean      8000 128    2       2       50                              0.138   
median    8000 128    8       1       50                              0.172   
                      2       1       100                             0.177   

                                                 F-measure   Goto  \
aggregate fmax n_mels ac_size std_bpm tightness                     
median    8000 128    8       2       100            0.353  0.078   
mean      8000 128    2       2       50             0.366  0.055   
median    8000 128    8       1       50             0.361  0.078   
                      2       1       100            0.356  0.083   

                                                 Information gain  P-score  \
aggregate fmax n_mels ac_size std_bpm tightness                              
median    8000 128    8       2       100                   0.176    0.480   
mean      8000 128    2       2       50                    0.164    0.476   
median    8000 128    8       1       50                    0.174    0.493   
                      2       1       100                   0.172    0.493   

                                                 index  
aggregate fmax n_mels ac_size std_bpm tightness         
median    8000 128    8       2       100          108  
mean      8000 128    2       2       50           108  
median    8000 128    8       1       50           108  
                      2       1       100          108

In [19]:
smc_scores.loc[best_igain]

Any Metric Level Continuous          0.173
Any Metric Level Total               0.316
Cemgil                               0.238
Cemgil Best Metric Level             0.305
Correct Metric Level Continuous      0.105
Correct Metric Level Total           0.172
F-measure                            0.353
Goto                                 0.078
Information gain                     0.176
P-score                              0.480
index                              108.000
Name: (median, 8000, 128, 8, 2.0, 100), dtype: float64

In [20]:
smc_scores.loc[best_fmeas]

Any Metric Level Continuous          0.137
Any Metric Level Total               0.315
Cemgil                               0.249
Cemgil Best Metric Level             0.321
Correct Metric Level Continuous      0.072
Correct Metric Level Total           0.138
F-measure                            0.366
Goto                                 0.055
Information gain                     0.164
P-score                              0.476
index                              108.000
Name: (mean, 8000, 128, 2, 2.0, 50), dtype: float64

In [21]:
smc_scores.loc[best_amlt]

Any Metric Level Continuous          0.155
Any Metric Level Total               0.334
Cemgil                               0.243
Cemgil Best Metric Level             0.303
Correct Metric Level Continuous      0.097
Correct Metric Level Total           0.172
F-measure                            0.361
Goto                                 0.078
Information gain                     0.174
P-score                              0.493
index                              108.000
Name: (median, 8000, 128, 8, 1.0, 50), dtype: float64

In [22]:
smc_scores.loc[best_cmlt]

Any Metric Level Continuous          0.163
Any Metric Level Total               0.328
Cemgil                               0.240
Cemgil Best Metric Level             0.299
Correct Metric Level Continuous      0.107
Correct Metric Level Total           0.177
F-measure                            0.356
Goto                                 0.083
Information gain                     0.172
P-score                              0.493
index                              108.000
Name: (median, 8000, 128, 2, 1.0, 100), dtype: float64

---

In [12]:
beatles_data = make_beatles_corpus('/home/bmcfee/data/beatles_iso/')

In [13]:
beatles_results = p_analyze_corpus(beatles_data)

[Parallel(n_jobs=3)]: Done   1 out of 179 | elapsed:  6.4min remaining: 1138.7min
[Parallel(n_jobs=3)]: Done  19 out of 179 | elapsed: 24.7min remaining: 208.1min
[Parallel(n_jobs=3)]: Done  37 out of 179 | elapsed: 51.6min remaining: 197.9min
[Parallel(n_jobs=3)]: Done  55 out of 179 | elapsed: 73.6min remaining: 166.0min
[Parallel(n_jobs=3)]: Done  73 out of 179 | elapsed: 89.3min remaining: 129.7min
[Parallel(n_jobs=3)]: Done  91 out of 179 | elapsed: 104.8min remaining: 101.4min
[Parallel(n_jobs=3)]: Done 109 out of 179 | elapsed: 122.5min remaining: 78.7min
[Parallel(n_jobs=3)]: Done 127 out of 179 | elapsed: 148.8min remaining: 60.9min
[Parallel(n_jobs=3)]: Done 145 out of 179 | elapsed: 168.0min remaining: 39.4min
[Parallel(n_jobs=3)]: Done 163 out of 179 | elapsed: 193.1min remaining: 19.0min
[Parallel(n_jobs=3)]: Done 179 out of 179 | elapsed: 211.2min finished
Skipping line 286: expected 2 fields, saw 3



Processing 01_-_I_Saw_Her_Standing_There.flac
Processing 02_-_Misery.flac
Processing 03_-_Anna_(Go_To_Him).flac
Processing 06_-_Ask_Me_Why.flacProcessing 04_-_Chains.flacProcessing 05_-_Boys.flac


Processing 10_-_Baby_It's_You.flacProcessing 07_-_Please_Please_Me.flacProcessing 08_-_Love_Me_Do.flac


Processing 13_-_There's_A_Place.flacProcessing 09_-_P._S._I_Love_You.flacProcessing 12_-_A_Taste_Of_Honey.flac


Processing 02_-_All_I've_Got_To_Do.flacProcessing 11_-_Do_You_Want_To_Know_A_Secret.flacProcessing 01_-_It_Won't_Be_Long.flac


Processing 04_-_Don't_Bother_Me.flacProcessing 14_-_Twist_And_Shout.flacProcessing 03_-_All_My_Loving.flac


Processing 08_-_Roll_Over_Beethoven.flacProcessing 05_-_Little_Child.flacProcessing 07_-_Please_Mister_Postman.flac


Processing 11_-_I_Wanna_Be_Your_Man.flacProcessing 06_-_Till_There_Was_You.flacProcessing 10_-_You_Really_Got_A_Hold_On_Me.flac


Processing 13_-_Not_A_Second_Time.flacProcessing 09_-_Hold_Me_Tight.flacProcessing 14_-_Money.flac


In [17]:
beatles_results.to_json('/home/bmcfee/git/librosa_parameters/beatles_beat_results.json', orient='records')

In [23]:
beatles_results = pd.read_json('/home/bmcfee/git/librosa_parameters/beatles_beat_results.json', orient='records')

In [24]:
beatles_scores = beatles_results.groupby(['aggregate', 'fmax', 'n_mels', 'ac_size', 'std_bpm', 'tightness']).mean()

In [25]:
beatles_scores

Any Metric Level Continuous  \
aggregate fmax  n_mels ac_size std_bpm tightness                                
mean      8000  32     2       0.5     50                               0.492   
                                       100                              0.575   
                                       400                              0.591   
                               1.0     50                               0.530   
                                       100                              0.620   
                                       400                              0.645   
                               2.0     50                               0.514   
                                       100                              0.595   
                                       400                              0.627   
                       4       0.5     50                               0.492   
                                       100                              0.575   
                                       400                              0.591   
                               1.0     50                               0.530   
                                       100                              0.620   
                                       400                              0.645   
                               2.0     50                               0.514   
                                       100                              0.595   
                                       400                              0.627   
                       8       0.5     50                               0.492   
                                       100                              0.575   
                                       400                              0.591   
                               1.0     50                               0.530   
                                       100                              0.620   
                                       400                              0.645   
                               2.0     50                               0.514   
                                       100                              0.595   
                                       400                              0.627   
                64     2       0.5     50                               0.539   
                                       100                              0.587   
                                       400                              0.600   
                               1.0     50                               0.578   
                                       100                              0.634   
                                       400                              0.656   
                               2.0     50                               0.577   
                                       100                              0.624   
                                       400                              0.648   
                       4       0.5     50                               0.539   
                                       100                              0.587   
                                       400                              0.600   
                               1.0     50                               0.578   
                                       100                              0.634   
                                       400                              0.656   
                               2.0     50                               0.577   
                                       100                              0.624   
                                       400                              0.648   
                       8       0.5     50                               0.539   
                                       100                              0.587   
                                       400                              0.600   


In [26]:
best_igain_b = beatles_scores['Information gain'].argmax()
best_fmeas_b = beatles_scores['F-measure'].argmax()
best_amlt_b = beatles_scores['Any Metric Level Total'].argmax()
best_cmlt_b = beatles_scores['Correct Metric Level Total'].argmax()

In [27]:
print best_igain
print best_fmeas
print best_amlt
print best_cmlt
print
print best_igain_b
print best_fmeas_b
print best_amlt_b
print best_cmlt_b

(u'median', 8000, 128, 8, 2.0, 100)
(u'mean', 8000, 128, 2, 2.0, 50)
(u'median', 8000, 128, 8, 1.0, 50)
(u'median', 8000, 128, 2, 1.0, 100)

(u'median', 8000, 128, 2, 2.0, 400)
(u'mean', 8000, 128, 2, 2.0, 100)
(u'median', 8000, 128, 2, 1.0, 400)
(u'median', 11025, 128, 2, 0.5, 400)


In [28]:
good_scores = ['Any Metric Level Total', 'Information gain', 'F-measure', 'Correct Metric Level Total']

In [29]:
smc_scores.loc[[best_igain, best_fmeas, best_amlt]][good_scores] 

,,,,,,Any Metric Level Total,Information gain,F-measure,Correct Metric Level Total
aggregate,fmax,n_mels,ac_size,std_bpm,tightness,,,,
median,8000,128,8,2,100,0.316,0.176,0.353,0.172
mean,8000,128,2,2,50,0.315,0.164,0.366,0.138
median,8000,128,8,1,50,0.334,0.174,0.361,0.172


In [30]:
smc_scores.loc[[best_igain_b, best_fmeas_b, best_amlt_b, best_cmlt_b]][good_scores]

Any Metric Level Total  \
aggregate fmax  n_mels ac_size std_bpm tightness                           
median    8000  128    2       2.0     400                         0.298   
mean      8000  128    2       2.0     100                         0.312   
median    8000  128    2       1.0     400                         0.301   
          11025 128    2       0.5     400                         0.264   

                                                  Information gain  F-measure  \
aggregate fmax  n_mels ac_size std_bpm tightness                                
median    8000  128    2       2.0     400                   0.161      0.326   
mean      8000  128    2       2.0     100                   0.164      0.357   
median    8000  128    2       1.0     400                   0.160      0.329   
          11025 128    2       0.5     400                   0.147      0.306   

                                                  Correct Metric Level Total  
aggregate fmax  n_mels ac_size std_bpm tightness                              
median    8000  128    2       2.0     400                             0.155  
mean      8000  128    2       2.0     100                             0.145  
median    8000  128    2       1.0     400                             0.161  
          11025 128    2       0.5     400                             0.124

In [31]:
beatles_scores.loc[[best_igain, best_fmeas, best_amlt, best_cmlt]][good_scores]

Any Metric Level Total  \
aggregate fmax n_mels ac_size std_bpm tightness                           
median    8000 128    8       2       100                         0.787   
mean      8000 128    2       2       50                          0.799   
median    8000 128    8       1       50                          0.806   
                      2       1       100                         0.812   

                                                 Information gain  F-measure  \
aggregate fmax n_mels ac_size std_bpm tightness                                
median    8000 128    8       2       100                   0.492      0.721   
mean      8000 128    2       2       50                    0.479      0.743   
median    8000 128    8       1       50                    0.483      0.741   
                      2       1       100                   0.493      0.742   

                                                 Correct Metric Level Total  
aggregate fmax n_mels ac_size std_bpm tightness                              
median    8000 128    8       2       100                             0.578  
mean      8000 128    2       2       50                              0.626  
median    8000 128    8       1       50                              0.639  
                      2       1       100                             0.642

In [32]:
beatles_scores.loc[[best_igain_b, best_fmeas_b, best_amlt_b, best_cmlt_b]][good_scores]

Any Metric Level Total  \
aggregate fmax  n_mels ac_size std_bpm tightness                           
median    8000  128    2       2.0     400                         0.789   
mean      8000  128    2       2.0     100                         0.805   
median    8000  128    2       1.0     400                         0.813   
          11025 128    2       0.5     400                         0.813   

                                                  Information gain  F-measure  \
aggregate fmax  n_mels ac_size std_bpm tightness                                
median    8000  128    2       2.0     400                   0.498      0.722   
mean      8000  128    2       2.0     100                   0.492      0.746   
median    8000  128    2       1.0     400                   0.497      0.740   
          11025 128    2       0.5     400                   0.495      0.727   

                                                  Correct Metric Level Total  
aggregate fmax  n_mels ac_size std_bpm tightness                              
median    8000  128    2       2.0     400                             0.581  
mean      8000  128    2       2.0     100                             0.629  
median    8000  128    2       1.0     400                             0.643  
          11025 128    2       0.5     400                             0.651

In [37]:
beatles_results['fmax'].unique()

array([ 8000, 11025])

In [33]:
smc_scores.loc[[('mean', 11025, 128, 4, 1.0, 400),
                best_amlt,
                best_amlt_b,
                best_cmlt,
                best_cmlt_b,
               ('median', 8000, 128, 4, 1.0, 100),]][good_scores]

Any Metric Level Total  \
aggregate fmax  n_mels ac_size std_bpm tightness                           
mean      11025 128    4       1.0     400                         0.282   
median    8000  128    8       1.0     50                          0.334   
                       2       1.0     400                         0.301   
                                       100                         0.328   
          11025 128    2       0.5     400                         0.264   
          8000  128    4       1.0     100                         0.329   

                                                  Information gain  F-measure  \
aggregate fmax  n_mels ac_size std_bpm tightness                                
mean      11025 128    4       1.0     400                   0.146      0.313   
median    8000  128    8       1.0     50                    0.174      0.361   
                       2       1.0     400                   0.160      0.329   
                                       100                   0.172      0.356   
          11025 128    2       0.5     400                   0.147      0.306   
          8000  128    4       1.0     100                   0.172      0.356   

                                                  Correct Metric Level Total  
aggregate fmax  n_mels ac_size std_bpm tightness                              
mean      11025 128    4       1.0     400                             0.109  
median    8000  128    8       1.0     50                              0.172  
                       2       1.0     400                             0.161  
                                       100                             0.177  
          11025 128    2       0.5     400                             0.124  
          8000  128    4       1.0     100                             0.177

In [35]:
beatles_scores.loc[[(u'mean', 11025.0, 128, 4, 1.0, 400),
                    best_amlt,
                    best_amlt_b,
                    best_cmlt,
                    best_cmlt_b,
                   ('median', 8000, 128, 4, 1.0, 100),]][good_scores]

Any Metric Level Total  \
aggregate fmax  n_mels ac_size std_bpm tightness                           
mean      11025 128    4       1.0     400                         0.807   
median    8000  128    8       1.0     50                          0.806   
                       2       1.0     400                         0.813   
                                       100                         0.812   
          11025 128    2       0.5     400                         0.813   
          8000  128    4       1.0     100                         0.812   

                                                  Information gain  F-measure  \
aggregate fmax  n_mels ac_size std_bpm tightness                                
mean      11025 128    4       1.0     400                   0.491      0.734   
median    8000  128    8       1.0     50                    0.483      0.741   
                       2       1.0     400                   0.497      0.740   
                                       100                   0.493      0.742   
          11025 128    2       0.5     400                   0.495      0.727   
          8000  128    4       1.0     100                   0.493      0.742   

                                                  Correct Metric Level Total  
aggregate fmax  n_mels ac_size std_bpm tightness                              
mean      11025 128    4       1.0     400                             0.634  
median    8000  128    8       1.0     50                              0.639  
                       2       1.0     400                             0.643  
                                       100                             0.642  
          11025 128    2       0.5     400                             0.651  
          8000  128    4       1.0     100                             0.642

In [36]:
best_igain_b

(u'median', 8000, 128, 2, 2.0, 400)

In [37]:
all_results = pd.concat([smc_results, beatles_results])

In [38]:
all_scores = all_results.groupby(['aggregate', 'fmax', 'n_mels', 'ac_size', 'std_bpm', 'tightness']).mean()

In [39]:
best_igain_a = all_scores['Information gain'].argmax()
best_fmeas_a = all_scores['F-measure'].argmax()
best_amlt_a = all_scores['Any Metric Level Total'].argmax()
best_cmlt_a = all_scores['Correct Metric Level Total'].argmax()

In [40]:
all_scores.loc[[(u'mean', 11025.0, 128, 4, 1.0, 400),
                    best_cmlt,
                    best_cmlt_b,
                    best_cmlt_a,
               ('median', 8000, 128, 4, 1.0, 100),]]

Any Metric Level Continuous  \
aggregate fmax  n_mels ac_size std_bpm tightness                                
mean      11025 128    4       1.0     400                              0.385   
median    8000  128    2       1.0     100                              0.376   
          11025 128    2       0.5     400                              0.377   
          8000  128    2       1.0     100                              0.376   
                       4       1.0     100                              0.377   

                                                  Any Metric Level Total  \
aggregate fmax  n_mels ac_size std_bpm tightness                           
mean      11025 128    4       1.0     400                         0.519   
median    8000  128    2       1.0     100                         0.547   
          11025 128    2       0.5     400                         0.512   
          8000  128    2       1.0     100                         0.547   
                       4       1.0     100                         0.547   

                                                  Cemgil  \
aggregate fmax  n_mels ac_size std_bpm tightness           
mean      11025 128    4       1.0     400         0.381   
median    8000  128    2       1.0     100         0.395   
          11025 128    2       0.5     400         0.371   
          8000  128    2       1.0     100         0.395   
                       4       1.0     100         0.395   

                                                  Cemgil Best Metric Level  \
aggregate fmax  n_mels ac_size std_bpm tightness                             
mean      11025 128    4       1.0     400                           0.455   
median    8000  128    2       1.0     100                           0.461   
          11025 128    2       0.5     400                           0.443   
          8000  128    2       1.0     100                           0.461   
                       4       1.0     100                           0.461   

                                                  Correct Metric Level Continuous  \
aggregate fmax  n_mels ac_size std_bpm tightness                                    
mean      11025 128    4       1.0     400                                  0.292   
median    8000  128    2       1.0     100                                  0.295   
          11025 128    2       0.5     400                                  0.295   
          8000  128    2       1.0     100                                  0.295   
                       4       1.0     100                                  0.295   

                                                  Correct Metric Level Total  \
aggregate fmax  n_mels ac_size std_bpm tightness                               
mean      11025 128    4       1.0     400                             0.346   
median    8000  128    2       1.0     100                             0.387   
          11025 128    2       0.5     400                             0.362   
          8000  128    2       1.0     100                             0.387   
                       4       1.0     100                             0.387   

                                                  F-measure   Goto  \
aggregate fmax  n_mels ac_size std_bpm tightness                     
mean      11025 128    4       1.0     400            0.504  0.295   
median    8000  128    2       1.0     100            0.530  0.298   
          11025 128    2       0.5     400            0.496  0.295   
          8000  128    2       1.0     100            0.530  0.298   
                       4       1.0     100            0.530  0.298   

                                                  Information gain  P-score  \
aggregate fmax  n_mels ac_size std_bpm tightness                              
mean      11025 128    4       1.0     400                   0.302    0.586   
median    8000  128    2       1.0     100                   0.317    0.615   
          11025 1